<a href="https://colab.research.google.com/github/Sahil1694/Box-Office-Prediction/blob/main/Box_Office_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd
import io

# Upload the file
uploaded = files.upload()

# Get the file name
file_name = list(uploaded.keys())[0]

# Load the dataset into pandas DataFrame
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

# Display the first few rows
df.head()


Saving final_dataset.csv to final_dataset.csv


,Unnamed: 0,movie,year,production_budget,domestic_gross,foreign_gross,worldwide_gross,month,profit,profit_margin,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,0,Avatar,2009,425000000,760507625,2015837654,2776345279,12,2351345279,0.846921,...,0,0,0,0,0,1,0,0,0,0
1,1,Pirates of the Caribbean: On Stranger Tides,2011,410600000,241063875,804600000,1045663875,5,635063875,0.607331,...,0,0,0,0,0,0,0,0,0,0
2,2,Avengers: Age of Ultron,2015,330600000,459005868,944008095,1403013963,5,1072413963,0.764364,...,0,0,0,0,0,1,0,0,0,0
3,3,Avengers: Infinity War,2018,300000000,678815482,1369318718,2048134200,4,1748134200,0.853525,...,0,0,0,0,0,0,0,0,0,0
4,4,Justice League,2017,300000000,229024295,426920914,655945209,11,355945209,0.542645,...,0,0,0,0,0,1,0,0,0,0


In [2]:
df.isnull().sum()  # Check for missing values
df.fillna(0, inplace=True)  # Replace missing values with 0 (or use mean/median)


In [4]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', dayfirst=True)
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month


<ipython-input-4-5a8b87b8467c>:1: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', dayfirst=True)


In [5]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
df['original_language'] = label_enc.fit_transform(df['original_language'])


In [9]:
genre_columns = ['Action', 'Adventure', 'Comedy', 'Drama', 'Horror', 'Science Fiction']  # Corrected 'Sci-Fi'
df = pd.get_dummies(df, columns=genre_columns)



In [12]:
# Define base features
features = ['production_budget', 'domestic_gross', 'foreign_gross', 'month',
            'original_language', 'vote_average', 'vote_count']

# Extract correctly formatted genre columns (binary indicators with '_1' suffix)
genre_columns = [col for col in df.columns if any(g in col for g in ['Action', 'Adventure', 'Comedy',
                                                                      'Drama', 'Horror', 'Science Fiction']) and '_1' in col]

# Add genre columns to features
features += genre_columns

# Define target variable
target = 'worldwide_gross'

# Create feature matrix (X) and target vector (y)
X = df[features]
y = df[target]

# Verify the feature selection
print("Selected Features:", X.columns)
print("Target:", target)


Selected Features: Index(['production_budget', 'domestic_gross', 'foreign_gross', 'month',
       'original_language', 'vote_average', 'vote_count', 'Action_1',
       'Adventure_1', 'Comedy_1', 'Drama_1', 'Horror_1', 'Science Fiction_1'],
      dtype='object')
Target: worldwide_gross


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [33]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [34]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [35]:
y_pred = model.predict(X_test)
rf_y_pred = rf_model.predict(X_test)


In [37]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np  # For square root calculation

def evaluate_model(y_true, y_pred, model_name):
    print(f"Model: {model_name}")
    print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_true, y_pred):.2f}")
    print(f"Mean Squared Error (MSE): {mean_squared_error(y_true, y_pred):.2f}")
    print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_true, y_pred)):.2f}")  # Manually computing RMSE
    print(f"R² Score: {r2_score(y_true, y_pred):.4f}\n")

# Call function
evaluate_model(y_test, y_pred, "Linear Regression")
evaluate_model(y_test, rf_y_pred, "Random Forest Regressor")



Model: Linear Regression
Mean Absolute Error (MAE): 67764186.04
Mean Squared Error (MSE): 13426691229976570.00
Root Mean Squared Error (RMSE): 115873600.23
R² Score: 0.6998

Model: Random Forest Regressor
Mean Absolute Error (MAE): 49078933.67
Mean Squared Error (MSE): 8403113099004089.00
Root Mean Squared Error (RMSE): 91668495.67
R² Score: 0.8121



In [27]:
print(X_train.head())
print(y_train.head())


      production_budget  domestic_gross  foreign_gross  month  \
199           110000000       324591735      462088822      5   
700            30000000        56443120       94364052     11   
526            42000000        45852178       11999999      5   
1223           10000000         4286367        1802733      6   
1604            1500000         7690044        6554887      3   

      original_language  vote_average  vote_count  Action_1  Adventure_1  \
199                   3           7.5        8773      True         True   
700                   3           7.3        2715     False        False   
526                   3           5.3         495      True        False   
1223                  3           5.2         231     False        False   
1604                  3           6.2         667     False        False   

      Comedy_1  Drama_1  Horror_1  Science Fiction_1  
199       True    False     False              False  
700      False     True     False         

In [28]:
#Need TO Run first
features = ['production_budget', 'month', 'original_language',
            'vote_average', 'vote_count'] + genre_columns

X = df[features]
y = df['worldwide_gross']


In [29]:
X[genre_columns] = X[genre_columns].astype(int)


<ipython-input-29-de68fb982545>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[genre_columns] = X[genre_columns].astype(int)


In [30]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, q=4, duplicates='drop')
)


In [32]:
print(f"Unique worldwide_gross values: {df['worldwide_gross'].nunique()}")


Unique worldwide_gross values: 1505
